In [47]:
import pandas as pd
import numpy as np
import re
from glob import glob
import regex
from googletrans import Translator


In [33]:
global data_dir, output_dir
data_dir = '../../data/original/external/schmoch/'
output_dir = '../../data/processed/external/schmoch/'


In [34]:
df_dict = pd.read_excel(f'{data_dir}ipc_technology.xlsx', 
                        sheet_name=None, 
                        engine='openpyxl', 
                        header=6, 
                        usecols=['Field_number', 'Field_en', 'IPC_code'])
df_dict.keys()

dict_keys(['35 Fields of Technology', 'Energy Technology'])

In [35]:
df_35 = df_dict['35 Fields of Technology'].copy()
df_35['IPC_code'] = df_35['IPC_code'].str.replace('%', '')
df_35


,Field_number,Field_en,IPC_code
0,1,"Electrical machinery, apparatus, energy",F21H
1,1,"Electrical machinery, apparatus, energy",F21K
2,1,"Electrical machinery, apparatus, energy",F21L
3,1,"Electrical machinery, apparatus, energy",F21S
4,1,"Electrical machinery, apparatus, energy",F21V
...,...,...,...
765,35,Civil engineering,E21B
766,35,Civil engineering,E21C
767,35,Civil engineering,E21D
768,35,Civil engineering,E21F


In [36]:
df_35.to_csv(f'{output_dir}35.csv', 
             sep=',', 
             encoding='utf-8', 
             index=False)


In [45]:
description_str = '''1. Electrical machinery, apparatus, energy: the field primarily covers the non-electronic part of electrical engineering, for instance, the generation, conversion and distribution of electric power, electric machines but also basic electric elements such as resistors, magnets, capacitors, lamps or cables. This field is often associated with “traditional” electrical engineering, but the high patent activity shows that technological innovation is still very important.
2. Audio-visual technology: audio-visual technology is largely equivalent to consumer electronics. The relevant IPC codes primarily refer to technologies and only sometimes products are directly addressed (H04R Loudspeakers ..., H04S Stereophonic systems)
3. Telecommunications: telecommunications is a very broad field covering a variety of techniques and products. The IPC codes are often quite technology-oriented, so that it is difficult to separate relevant product/applications areas such as mobile communication in a clear-cut field.
4. Digital communication: in the ISI-OST-INPI classification, this field was part of telecommunications. At present, it is a self-contained technology at the border between telecommunications and computer technology. A core application of this technology is the internet.
5. Basic communication processes: in the ISI-OST-INPI classification, this field was part of telecommunications. It covers very basic technologies such as oscillation, modulation, resonant circuits, impulse technique, coding/decoding. These techniques are used in telecommunications, computer technology, measurement, control. However, the explicit link to these fields by multiple classification is moderate, in the case of telecommunications 2.4 percent. So the definition as a separate field is justified. However, with 0.9 percent of all applications in 2005, it is the smallest fields of the present version of the classification.
6. Computer technology: this field is the largest of the proposed classification with 6.4 percent of all applications in 2005. Its size is already reduced by extracting field 7. The core area of C06F (Electrical digital processing) is defined in a very technical way (Arrangement for programme control, methods and arrangements for data conversion ...), so that a further break-down is difficult. It may be possible to separate specific application fields such as image data processing, recognition of data or speech analysis, but then these special fields may become too small.
7. IT methods for management: a major improvement of IPC8 is the introduction of the sub-class G06Q “Data processing methods, specially adapted for administrative, commercial, financial, managerial, supervisory or forecasting purposes”. This field represents software for these special purposes. In most countries, business methods are not patentable, but if they are admitted, they are registered in this sub-class. In any case, the size of this field is relevant with 1.2 percent of all applications in 2005. A combination of the fields 3 to 7 represents information technology in general. As the overlap is limited, this can be done by simple addition. The correct way is to combine the fields without double counting (unit)
8. Semiconductors: the field comprises semiconductors including methods for their production. Integrated circuits or photovoltaic elements belong to this field. The field includes microstructural technology (B81), as the number of applications in this sub-field is too small for a separate field.
9. Optics: this field covers all parts of traditional optical elements and apparatus, but also laser beam sources. In recent years new optical technologies such as optical switching have become more relevant.
10. Measurement: this field covers a broad variety of different techniques and applications. It would be possible to differentiate special sub-fields such as measuring of mechanical properties (length, oscillation, speed ...), but these sub-fields are generally too small.
11. Analysis of biological materials: this is the largest sub-field of “measurement” and was defined as a separate field. It primarily refers to the analysis of blood for medical purposes. In many cases, biotechnological methods are addressed.
12. Control: In the ISI-OST-INPI classification, this field was part of measuring & control. In recent years the part of control has become quantitatively more important, so that an independent field is justified. The field covers elements for controlling and regulating electrical and non-electrical systems and referring test arrangements, traffic control or signalling systems etc.
13. Medical technology: Medical technology is generally associated with high technology. However, a large part of the class A61 refers to less sophisticated products and technologies such as operating tables, massage devices, bandages etc. These less complex sub-fields represent a large number of patent applications, and the total field is the second largest of the suggested classification with 6.3 percent of all applications in 2005.
14. Organic fine chemistry: without further limitations, the applications in organic chemistry primarily refer to pharmaceuticals. More than 40 percent of the applications have an additional code in pharmaceuticals. As such a large overlap of fields is less appropriate for a classification system, all documents with co-classification in A61K were excluded. The major exception is the group A61K-008, which refers to cosmetics.
15. Biotechnology: biotechnology is defined as a separate field, although it is linked to a variety of different applications. Like organic chemistry or computer technology, it is a crosscutting or generic technology. However, the overlap with pharmaceuticals is too large, with a share of nearly 30 percent. Therefore, as in organic chemistry, applications with explicit co-classification in A61K are excluded.
16. Pharmaceuticals: this field refers to an area of application, not a technology. However, the key sub-class A61K is primarily organized by technologies (e.g., medicinal preparations containing inorganic active ingredients …). Cosmetics are explicitly excluded from the field; these represent about 10 percent of all applications classified in A61K.
17. Macromolecular chemistry, polymers: this field contains the chemical aspects of polymers. Machines for producing articles from plastics are classified in B29 and not included.
18. Food chemistry: this field represents 1.3 percent of the applications in 2005 and is one of the smallest fields in this classification. However, the growth of this field is remarkable, so that a higher weight can be assumed for the next years. Machines for food production are not included, but classified as part of field 28 (other special machines)
19. Basic materials chemistry: This field primarily covers typical mass chemicals such as herbicides, fertilisers, paints, petroleum, gas, detergents etc.
20. Materials, metallurgy: This field covers all types of metals, ceramics, glass or processes for the manufacture of steel.
21. Surface technology, coating: The coating of metals, generally with advanced methods represents the core of this field (C23). Furthermore it covers electrolytic processes, crystal growth and apparatus for applying liquids to surfaces. This field may be qualified as the high-tech part of field 20.
22. Micro-structure and nano-technology: This field covers micro-structural devices or systems, including at least one essential element or formation characterised by its very small size. It includes nano-structures having specialised features directly related to their size.
23. Chemical engineering: This field covers technologies at the borderline of chemistry and engineering. It refers to apparatus and processes for the industrial production of chemicals. Some of these processes may be classified as physical ones.
24. Environmental technology: This field covers a variety of different technologies and applications, in particular filters, waste disposal, water cleaning (a quite large area), gas-flow silencers and exhaust apparatus, waste combustion or noise absorption walls. However, it is not possible to define measuring of environmental pollution by IPC codes in a comprehensive way.
25. Handling: This field comprises elevators, cranes or robots, but also packaging devices. So in terms of research intensity, the field is quite heterogeneous.
26. Machine tools: The field is dominated by patent applications referring to turning, boring, grinding, soldering or cutting with a focus on metals.
27. Engines, pumps, turbines: This field covers non-electrical engines for all types of applications. In quantitative terms, applications for automobiles dominate.
28. Textile and paper machines: The fields 27 and 28 cover machines for specific production purposes. Textile and food machines represent the most relevant part of these machines and are classified separately.
29. Other special machines: see field 26.
30. Thermal processes and apparatus: The field covers applications such as steam generation, combustion, heating, refrigeration, cooling or heat exchange.
31. Mechanical elements: The field covers fluid-circuit elements, joints, shafts, couplings, valves, pipe-line systems or mechanical control devices. The focus is on engineering elements of machines such as joints or couplings.
32. Transport: the field covers all types of transport technology and applications with dominance of automotive technology. In principle, a separation of rail traffic and air traffic would be feasible, but the associated fields would be too small. In both cases, this is due to a low propensity to patent. The samples are quite small and not representative of the total technological activities in these sub-fields.
33. Furniture, games: this field represents the main parts of consumer goods in terms of the number of patent applications. The other consumer goods are a mix of many different technologies, all of them with low quantitative weight. Therefore a further differentiation is not useful. Even furniture and games combined comprise not more than 2.3 percent of all applications in 2005.
34. Other consumer goods: this field primarily represents less research-intensive sub-fields.
35. Civil engineering: the field covers construction of roads and buildings as well as elements of buildings such as locks, plumbing installations or strongrooms for valuables. A special part refers to mining which may be important for some countries. In general, the importance of this field is so low that the definition of a separate field is not justified.
'''
description_list = description_str.split('\n')
description_dict = {
                    'field': [field.split(':')[0].split('. ')[-1] for field in description_list][:-1],
                    'description': [description.split(':')[-1][1:] for description in description_list][:-1]
                    }
description_en_df = pd.DataFrame(description_dict)
description_en_df.index += 1
description_en_df.to_csv(f'{output_dir}35_description_en.csv', 
                        sep=',', 
                        encoding='utf-8', 
                        index=False)


In [51]:
description_ja_df = description_en_df.copy()
translator =Translator()
for i, row in description_ja_df.iterrows():
     description_ja_df.loc[i, 'field'] = translator.translate(text=row['field'], dest="ja").text
     description_ja_df.loc[i, 'description'] = translator.translate(text=row['description'], dest="ja").text
description_ja_df.to_csv(f'{output_dir}35_description_ja.csv', 
                         sep=',',
                         encoding='utf-8',
                         index=False)


,field,description
1,電気機械、装置、エネルギー,このフィールドは、主に電気工学の非電子部分をカバーしています。たとえば、電力、電気機械の生成...
2,オーディオビジュアルテクノロジー,視聴覚テクノロジーは、大部分が家電に相当します。関連するIPCコードは主にテクノロジーを指し...
3,通信,電気通信は、さまざまな技術や製品をカバーする非常に広範な分野です。IPCコードは多くの場合、...
4,デジタル通信,ISI-ast-inpi分類では、この分野は通信の一部でした。現在、それは電気通信とコンピュ...
5,基本的な通信プロセス,ISI-ast-inpi分類では、この分野は通信の一部でした。振動、変調、共振回路、衝動技術...
6,コンピューターテクノロジー,このフィールドは、2005年にすべてのアプリケーションの6.4％を持つ提案された分類の最大で...
7,管理のためのITメソッド,IPC8の大幅な改善は、サブクラスのG06Q「データ処理方法の導入であり、特に管理、商業、財...
8,半導体,このフィールドは、生産方法を含む半導体で構成されています。統合された回路または太陽光発電要素...
9,光学,このフィールドは、従来の光学要素と装置のすべての部分をカバーしていますが、レーザービーム源も...
10,測定,このフィールドは、さまざまなさまざまな技術とアプリケーションをカバーしています。機械的特性（...


In [58]:
description_df = pd.concat([description_en_df.rename(columns={'field':'field_en', 'description':'description_en'}), 
                            description_ja_df.rename(columns={'field':'field_ja', 'description':'description_ja'})], 
                           axis='columns')[['field_en', 'field_ja', 'description_en', 'description_ja']]
description_df.to_csv(f'{output_dir}35_description.csv',
                      sep=',',
                      encoding='utf-8',
                      index=False)

In [59]:
description_df

,field_en,field_ja,description_en,description_ja
1,"Electrical machinery, apparatus, energy",電気機械、装置、エネルギー,the field primarily covers the non-electronic ...,このフィールドは、主に電気工学の非電子部分をカバーしています。たとえば、電力、電気機械の生成...
2,Audio-visual technology,オーディオビジュアルテクノロジー,audio-visual technology is largely equivalent ...,視聴覚テクノロジーは、大部分が家電に相当します。関連するIPCコードは主にテクノロジーを指し...
3,Telecommunications,通信,telecommunications is a very broad field cover...,電気通信は、さまざまな技術や製品をカバーする非常に広範な分野です。IPCコードは多くの場合、...
4,Digital communication,デジタル通信,"in the ISI-OST-INPI classification, this field...",ISI-ast-inpi分類では、この分野は通信の一部でした。現在、それは電気通信とコンピュ...
5,Basic communication processes,基本的な通信プロセス,"in the ISI-OST-INPI classification, this field...",ISI-ast-inpi分類では、この分野は通信の一部でした。振動、変調、共振回路、衝動技術...
6,Computer technology,コンピューターテクノロジー,this field is the largest of the proposed clas...,このフィールドは、2005年にすべてのアプリケーションの6.4％を持つ提案された分類の最大で...
7,IT methods for management,管理のためのITメソッド,a major improvement of IPC8 is the introductio...,IPC8の大幅な改善は、サブクラスのG06Q「データ処理方法の導入であり、特に管理、商業、財...
8,Semiconductors,半導体,the field comprises semiconductors including m...,このフィールドは、生産方法を含む半導体で構成されています。統合された回路または太陽光発電要素...
9,Optics,光学,this field covers all parts of traditional opt...,このフィールドは、従来の光学要素と装置のすべての部分をカバーしていますが、レーザービーム源も...
10,Measurement,測定,this field covers a broad variety of different...,このフィールドは、さまざまなさまざまな技術とアプリケーションをカバーしています。機械的特性（...


---


In [6]:
# energyを細かく見る場合
df = pd.concat([value for value in df_dict.values()], 
               axis="index", 
               ignore_index=True)\
    .drop_duplicates(subset=['IPC_code'], keep='first')
df['IPC_code'] = df['IPC_code'].str.replace('%', '').str.replace('#', '')
df['IPC_class'] = df['IPC_code'].str.replace('%', '').str.replace('#', '').str[:3]
df['IPC_subclass'] = df['IPC_code'].str.replace('%', '').str.replace('#', '').str[:4]
df


,Field_number,Field_en,IPC_code,IPC_class,IPC_subclass
0,1,"Electrical machinery, apparatus, energy",F21H,F21,F21H
1,1,"Electrical machinery, apparatus, energy",F21K,F21,F21K
2,1,"Electrical machinery, apparatus, energy",F21L,F21,F21L
3,1,"Electrical machinery, apparatus, energy",F21S,F21,F21S
4,1,"Electrical machinery, apparatus, energy",F21V,F21,F21V
...,...,...,...,...,...
832,94,Hydro energy,B63H 19/04,B63,B63H
833,94,Hydro energy,E02B 9/00,E02,E02B
834,94,Hydro energy,E02B 9/02,E02,E02B
835,94,Hydro energy,E02B 9/04,E02,E02B


In [7]:
df[(df['IPC_code'] != df['IPC_subclass']) & (df['IPC_code'] != df['IPC_class'])]

,Field_number,Field_en,IPC_code,IPC_class,IPC_subclass
33,2,Audio-visual technology,H04N 3/,H04,H04N
34,2,Audio-visual technology,H04N 5/,H04,H04N
35,2,Audio-visual technology,H04N 7/,H04,H04N
36,2,Audio-visual technology,H04N 9/,H04,H04N
37,2,Audio-visual technology,H04N 11/,H04,H04N
...,...,...,...,...,...
832,94,Hydro energy,B63H 19/04,B63,B63H
833,94,Hydro energy,E02B 9/00,E02,E02B
834,94,Hydro energy,E02B 9/02,E02,E02B
835,94,Hydro energy,E02B 9/04,E02,E02B


In [8]:
for uni_str in df[(df['IPC_code'] != df['IPC_subclass']) & (df['IPC_code'] != df['IPC_class'])]['IPC_subclass'].unique():
    print('=======================================')
    display(df[df['IPC_code']==uni_str])
    display(df[df['IPC_subclass']==uni_str])

,Field_number,Field_en,IPC_code,IPC_class,IPC_subclass


,Field_number,Field_en,IPC_code,IPC_class,IPC_subclass
33,2,Audio-visual technology,H04N 3/,H04,H04N
34,2,Audio-visual technology,H04N 5/,H04,H04N
35,2,Audio-visual technology,H04N 7/,H04,H04N
36,2,Audio-visual technology,H04N 9/,H04,H04N
37,2,Audio-visual technology,H04N 11/,H04,H04N
38,2,Audio-visual technology,H04N 13/,H04,H04N
39,2,Audio-visual technology,H04N 15/,H04,H04N
40,2,Audio-visual technology,H04N 17/,H04,H04N
41,2,Audio-visual technology,H04N 19/,H04,H04N
42,2,Audio-visual technology,H04N 23/,H04,H04N


,Field_number,Field_en,IPC_code,IPC_class,IPC_subclass


,Field_number,Field_en,IPC_code,IPC_class,IPC_subclass
111,10,Measurement,G01N 1/,G01,G01N
112,10,Measurement,G01N 3/,G01,G01N
113,10,Measurement,G01N 5/,G01,G01N
114,10,Measurement,G01N 7/,G01,G01N
115,10,Measurement,G01N 9/,G01,G01N
116,10,Measurement,G01N 11/,G01,G01N
117,10,Measurement,G01N 13/,G01,G01N
118,10,Measurement,G01N 15/,G01,G01N
119,10,Measurement,G01N 17/,G01,G01N
120,10,Measurement,G01N 19/,G01,G01N


,Field_number,Field_en,IPC_code,IPC_class,IPC_subclass


...

,Field_number,Field_en,IPC_code,IPC_class,IPC_subclass
628,32,Transport,B60L,B60,B60L
825,92,Wind energy technology,B60L 8/00,B60,B60L


,Field_number,Field_en,IPC_code,IPC_class,IPC_subclass
658,32,Transport,B63H,B63,B63H


,Field_number,Field_en,IPC_code,IPC_class,IPC_subclass
658,32,Transport,B63H,B63,B63H
831,94,Hydro energy,B63H 19/02,B63,B63H
832,94,Hydro energy,B63H 19/04,B63,B63H


,Field_number,Field_en,IPC_code,IPC_class,IPC_subclass
744,35,Civil engineering,E02B,E02,E02B


,Field_number,Field_en,IPC_code,IPC_class,IPC_subclass
744,35,Civil engineering,E02B,E02,E02B
833,94,Hydro energy,E02B 9/00,E02,E02B
834,94,Hydro energy,E02B 9/02,E02,E02B
835,94,Hydro energy,E02B 9/04,E02,E02B
836,94,Hydro energy,E02B 9/06,E02,E02B
